<a href="https://colab.research.google.com/github/sebinsaji007/market-pattern-recognition/blob/main/doubletop_much_better.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import yfinance as yf

In [16]:
ticker = 'BTC-USD'
data = yf.download("BTC-USD", start="2023-08-21", end="2023-09-24",  interval = "60m")

[*********************100%%**********************]  1 of 1 completed


In [17]:
import yfinance as yf
import numpy as np
import pandas as pd


def average_true_range_corrected(data, period=14):
    """Calculate the Average True Range (ATR) for a DataFrame."""
    # Calculate true range for each row
    high_minus_low = data['High'] - data['Low']
    high_minus_close_prev = abs(data['High'] - data['Close'].shift(1))
    low_minus_close_prev = abs(data['Low'] - data['Close'].shift(1))

    data['TR'] = pd.concat([high_minus_low, high_minus_close_prev, low_minus_close_prev], axis=1).max(axis=1)

    # Calculate ATR
    data['ATR'] = data['TR'].rolling(window=period).mean()
    data.drop(columns=['TR'], inplace=True)  # Drop TR column after calculating ATR
    return data

# Adjusting the main function to use the corrected ATR function
def detect_double_top_with_volume_confirmation(data):
    detected_patterns = []

    # Calculate ATR for the data
    data = average_true_range_corrected(data)

    for i in range(2, len(data) - 13):  # Adjusting the range to ensure we have at least 10 candles after the detected pattern
        first_top = data['High'].iloc[i-2]
        first_top_volume = data['Volume'].iloc[i-2]

        neckline = data['Low'].iloc[i-1]

        second_top = data['High'].iloc[i]
        second_top_volume = data['Volume'].iloc[i]

        atr_value = data['ATR'].iloc[i]

        # Using ATR to adjust the tolerance dynamically and checking volume confirmation
        if abs(first_top - second_top) < atr_value and second_top_volume < first_top_volume:
            if data['Close'].iloc[i-2] > neckline and data['Open'].iloc[i-2] > neckline:
                if data['Close'].iloc[i] > neckline and data['Open'].iloc[i] > neckline:
                    if data['Close'].iloc[i+1] < neckline:
                        neckline_broken_price = data['Close'].iloc[i+1]
                        neckline_broken_date = data.index[i+1]
                        lowest_next_10 = min(data['Low'].iloc[i+2:i+12])
                        highest_next_10 = max(data['High'].iloc[i+2:i+12])
                    elif data['Close'].iloc[i+2] < neckline:
                        neckline_broken_price = data['Close'].iloc[i+2]
                        neckline_broken_date = data.index[i+2]
                        lowest_next_10 = min(data['Low'].iloc[i+3:i+13])
                        highest_next_10 = max(data['High'].iloc[i+3:i+13])
                    else:
                        continue

                    percentage_drop = ((lowest_next_10 - neckline_broken_price) / neckline_broken_price) * 100
                    percentage_rise = ((highest_next_10 - neckline_broken_price) / neckline_broken_price) * 100

                    detected_patterns.append({
                        'First Top Date': data.index[i-2],
                        'First Top Price': first_top,
                        'First Top Volume': first_top_volume,
                        'Neckline Date': data.index[i-1],
                        'Neckline Price': neckline,
                        'Second Top Date': data.index[i],
                        'Second Top Price': second_top,
                        'Second Top Volume': second_top_volume,
                        'Neckline Broken Date': neckline_broken_date,
                        'Neckline Broken Price': neckline_broken_price,
                        'Lowest of Next 10 Candles': lowest_next_10,
                        'Highest of Next 10 Candles': highest_next_10,
                        'Percentage Drop': percentage_drop,
                        'Percentage Rise': percentage_rise
                    })

    return pd.DataFrame(detected_patterns)

# Running the function on the sample data to demonstrate the volume confirmation feature
df_with_volume_confirmation = detect_double_top_with_volume_confirmation(data)
df_with_volume_confirmation.head()

,First Top Date,First Top Price,First Top Volume,Neckline Date,Neckline Price,Second Top Date,Second Top Price,Second Top Volume,Neckline Broken Date,Neckline Broken Price,Lowest of Next 10 Candles,Highest of Next 10 Candles,Percentage Drop,Percentage Rise
0,2023-08-23 06:00:00,26155.927734,299508736,2023-08-23 07:00:00,26011.603516,2023-08-23 08:00:00,26084.464844,0,2023-08-23 10:00:00,25955.431641,25804.998047,26786.898438,-0.579584,3.203440
1,2023-08-23 07:00:00,26117.300781,238130176,2023-08-23 08:00:00,26014.511719,2023-08-23 09:00:00,26040.070312,21294080,2023-08-23 10:00:00,25955.431641,25804.998047,26786.898438,-0.579584,3.203440
2,2023-08-24 08:00:00,26481.482422,63758336,2023-08-24 09:00:00,26428.111328,2023-08-24 10:00:00,26462.531250,0,2023-08-24 12:00:00,26381.816406,25914.925781,26406.154297,-1.769744,0.092253
3,2023-08-24 09:00:00,26554.910156,261795840,2023-08-24 10:00:00,26417.800781,2023-08-24 11:00:00,26522.373047,0,2023-08-24 12:00:00,26381.816406,25914.925781,26406.154297,-1.769744,0.092253
4,2023-08-25 03:00:00,26129.994141,138561536,2023-08-25 04:00:00,26022.707031,2023-08-25 05:00:00,26068.664062,0,2023-08-25 06:00:00,26015.769531,25786.812500,26248.103516,-0.880070,0.893051


In [ ]:
data.head(30)

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-09-21 00:00:00,27129.839844,27152.939453,27053.494141,27053.494141,27053.494141,0
2023-09-21 01:00:00,27051.994141,27051.994141,26984.078125,27028.373047,27028.373047,67324928
2023-09-21 02:00:00,27027.544922,27054.855469,26944.951172,26970.097656,26970.097656,0
2023-09-21 03:00:00,26971.574219,27023.371094,26971.212891,27022.046875,27022.046875,0
2023-09-21 04:00:00,27036.419922,27106.505859,27003.498047,27095.580078,27095.580078,8537088
2023-09-21 05:00:00,27098.591797,27098.591797,27045.884766,27075.259766,27075.259766,0
2023-09-21 06:00:00,27074.275391,27074.275391,27017.472656,27069.623047,27069.623047,0
2023-09-21 07:00:00,27072.947266,27072.947266,26983.216797,26987.417969,26987.417969,0
2023-09-21 08:00:00,26988.939453,27011.835938,26918.255859,26927.673828,26927.673828,31377408


In [ ]:
df.to_csv('new.csv')